This is the code for data collection, we refer to the paper for the overview. In short, we do this in following way:
- The gelsight mini is mounted at the end effector of the robot
- An object is placed under the robot, such that the robot can touch the object from the top
- We drop the end effector 1 mm at a time. When a threshold in both the SSIM and Marker Displacement has passed, contact is detected
- From contact, we take 8 images at equally positioned depths of 0.25mm, untill the end effector reaches 2mm under the contact detection height
- The above procedure is repeated 40 times per object, with varying yaw and position for more robust and varying data

# 1) Import Packages and Set Directories for Data

In [ ]:
import time
import os
import random
import csv
from xarm.wrapper import XArmAPI
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [ ]:
save_dir = "Data"
os.makedirs(save_dir, exist_ok=True)

#Close the block underneath if you want (which is likely) to append your data to the existing dataframe
with open(os.path.join(save_dir, "Data_Overview.csv"), 'w', newline='') as f:
   writer = csv.writer(f)
   writer.writerow(['Object','Pose_Number', 'Contact', 'Hardness_Level', 'SSIM', 'Marker_Displacement', 'Z'])

# 2) Contact Definition
Based on two criteria:
- marker displacement should be >=2
- SSIM should be <0.96
The combination minimizes false contacts while being sensitive to soft materials

In [ ]:
def label_markers(image_path, plot=False):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(img, 40, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)

    # Use of connected component labelling from course of Image Processing
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(cleaned, connectivity=8)
    h, w = binary.shape
    valid_centroids = []

    for i in range(1, num_labels):  # Skip background, therefore start from 1
        x, y, bw, bh, area = stats[i]
        cx, cy = centroids[i]
        cx, cy = int(cx), int(cy)

        if 10 < area < 750 and 75 < x < w - 75 and 75 < y < h - 75:
            new_centroid = np.array([cx, cy])
            merged = False

            for j, existing in enumerate(valid_centroids):
                if np.linalg.norm(existing - new_centroid) < 20:
                    valid_centroids[j] = (existing + new_centroid) / 2
                    merged = True

            if not merged:
                if len(valid_centroids) != 63: #because I noticed right under has black dot possibly that should not be in there
                    valid_centroids.append(new_centroid)

    # Convert to int tuples for drawing
    valid_centroids = [tuple(map(int, c)) for c in valid_centroids]

    if plot:
        img_debug = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        for (cx, cy) in valid_centroids:
            cv2.circle(img_debug, (cx, cy), 4, (0, 255, 0), -1)

        plt.imshow(cv2.cvtColor(img_debug, cv2.COLOR_BGR2RGB))
        plt.title(f"Detected markers: {len(valid_centroids)}")
        plt.axis("off")
        plt.show()

    return np.array(valid_centroids)

def match_unique_pairs(centroids_ref, centroids_def, max_distance=40):
    # Compute pairwise distance matrix
    dist_matrix = cdist(centroids_def, centroids_ref)

    # Mask out impossible matches
    dist_matrix[dist_matrix > max_distance] = 100

    # Solve assignment: linear_sum_assignment can find optimal way to assign the points while minimizing costs,
    # This makes sure that each displaced marker is connected to its origin, and not just to the closest marker
    # in the neighbourhood, for reference see: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html
    row_idx, col_idx = linear_sum_assignment(dist_matrix)

    # Filter by max distance again (in case no match was acceptable)
    valid = dist_matrix[row_idx, col_idx] < max_distance

    matched_def = centroids_def[row_idx[valid]]
    matched_ref = centroids_ref[col_idx[valid]]

    return matched_ref, matched_def

def calculate_marker_displacement(image_path1, image_path2):
    centroids_ref = label_markers(image_path1)
    centroids_def = label_markers(image_path2)

    matched_ref, matched_def = match_unique_pairs(centroids_ref, centroids_def)

    displacement_vectors = matched_def - matched_ref
    magnitudes = np.linalg.norm(displacement_vectors, axis=1)

    return np.mean(magnitudes)

# 3) Get robot and camera ready

In [ ]:
def record_and_save_image(path):
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    frame = cv2.resize(frame, (512, 512))
    cv2.imwrite(path, frame)

In [ ]:
arm = XArmAPI('192.168.1.117')  # Replace with your robot's IP address
arm.connect()
arm.clean_warn()
arm.clean_error()
arm.motion_enable(enable=True)
arm.set_mode(0)      # Position mode
arm.set_state(0)     # Ready state

In [ ]:
initial_pose = arm.get_position()[1]
z_init = initial_pose[2]
arm.set_position(z=z_init, roll = 180, yaw = 90, pitch= 0 )
print(initial_pose)

In [ ]:
pose = arm.get_position()[1]

# 4) Collect Data